In [ ]:
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 12.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers datasets accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00


In [ ]:
#Install Libraries
!pip install -q tensorflow bert-for-tf2 tokenizers
!pip install transformers
!pip install tensorflow-text

In [ ]:
!pip install -U scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 51.9 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [ ]:
from transformers import AutoTokenizer, DistilBertModel
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig
import pandas
import numpy
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
import nltk
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input
from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
import itertools
from sklearn.utils import shuffle
from tensorflow.keras import regularizers
import sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# Import Data
from google.colab import drive
drive.mount('/content/drive')
X_test = pd.read_csv('/content/drive/MyDrive/BTAI/tweetsum_test.csv')
X_train = pd.read_csv('/content/drive/MyDrive/BTAI/tweetsum_train.csv')
X_val = pd.read_csv('/content/drive/MyDrive/BTAI/tweetsum_val.csv')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def clean(test,train,val):
  # Col for Word Count
  train['word_count']=train['sentence'].apply(lambda x:len(x.split()))
  test['word_count']=test['sentence'].apply(lambda x:len(x.split()))
  val['word_count']=val['sentence'].apply(lambda x:len(x.split()))

  #adding a new column to X_train "is_question" that flags true for question senteces and false for non-question sentences
  question_words = ["who", "what", "when", "where", "why", "how", "if"]
  train['is_question'] = (train['sentence'].str.endswith('?') | train['sentence'].str.lower().str.contains(fr'\b(?:{"|".join(question_words)})\b'))
  test['is_question'] = (train['sentence'].str.endswith('?') | train['sentence'].str.lower().str.contains(fr'\b(?:{"|".join(question_words)})\b'))
  val['is_question'] = (train['sentence'].str.endswith('?') | train['sentence'].str.lower().str.contains(fr'\b(?:{"|".join(question_words)})\b'))

  # # Drop sender, annot_id, dialog_id, and Unnamed: 0
  # train.drop(['sender', 'annot_id', 'dialog_id', 'Unnamed: 0'], axis = 1, inplace = True)
  # test.drop(['sender', 'annot_id', 'dialog_id', 'Unnamed: 0'], axis = 1, inplace = True)
  # val.drop(['sender', 'annot_id', 'dialog_id', 'Unnamed: 0'], axis = 1, inplace = True)

  return test,train,val

In [ ]:
train, test, val = clean(X_train, X_test, X_val)

In [ ]:
from transformers import AutoTokenizer, TFAutoModel
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig

import torch
import tensorflow as tf

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = TFAutoModel.from_pretrained('bert-base-uncased')

#DistilBertModel.from_pretrained("distilbert-base-uncased")

inputs = tokenizer("Hello, my dog is cute", return_tensors="tf")
outputs = model(inputs)

last_hidden_states = outputs.last_hidden_state


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [ ]:
BERT_Encodings=[]
for sentence in X_train["sentence"]:
  inputs=tokenizer(sentence,return_tensors="pt")
  #print(sentence)
  outputs=model(**inputs)
  BERT_Encodings.append(outputs.last_hidden_state)

BERT_Encoding_pd=pd.DataFrame()
BERT_Encoding_pd["Encoding"]=BERT_Encodings
fileName="BERT_Enconding.xlsx"
BERT_Encoding_pd.to_excel(fileName)

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(100,), dtype=tf.string, name='text')  # Assuming max sequence length is 100
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

preprocessed_text = tokenizer(repr(text_input), return_tensors="tf")
outputs = dbert_model(**preprocessed_text)
last_hidden_states = outputs.last_hidden_state

# Neural network layers
dropout_input = tf.keras.layers.Input(shape=(100,), dtype=tf.float32)  # Assuming max sequence length is 100
l = tf.keras.layers.Dropout(0.1, name="dropout")(dropout_input)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input, dropout_input], outputs=[l])

# Print model summary
model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 100)]                0         []                            
                                                                                                  
 dropout (Dropout)           (None, 100)                  0         ['input_1[0][0]']             
                                                                                                  
 text (InputLayer)           [(None, 100)]                0         []                            
                                                                                                  
 output (Dense)              (None, 1)                    101       ['dropout[0][0]']             
                                                                                              

In [ ]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy','f1-score','precision','recall'])

In [ ]:
train = pd.get_dummies(train, columns = ['annot_id', 'sender'])
test = pd.get_dummies(test, columns = ['annot_id', 'sender'])
val =pd.get_dummies(val, columns = ['annot_id', 'sender'])

In [ ]:
val.columns
features=[ 'sentence', 'word_count',
       'is_question', 'annot_id_0', 'annot_id_1', 'annot_id_2', 'annot_id_3',
       'annot_id_4', 'sender_agent', 'sender_customer']
eval='is_salient'

In [ ]:
train['is_salient']=train['is_salient'].apply(lambda x: 1 if x==True else 0)
val['is_salient']=val['is_salient'].apply(lambda x: 1 if x==True else 0)
test['is_salient']=test['is_salient'].apply(lambda x: 1 if x==True else 0)


In [ ]:
train.dropna(inplace=True)

In [ ]:
train[train["is_salient"]==1]

,Unnamed: 0,dialog_id,sentence,is_salient,word_count,is_question,annot_id_0,annot_id_1,annot_id_2,annot_id_3,annot_id_4,sender_agent,sender_customer
0,0,b065262210783596c1fe79466b8f8985,So neither my iPhone nor my Apple Watch are re...,1,22,False,1,0,0,0,0,0,1
1,1,b065262210783596c1fe79466b8f8985,So neither my iPhone nor my Apple Watch are re...,1,22,False,0,1,0,0,0,0,1
2,2,b065262210783596c1fe79466b8f8985,So neither my iPhone nor my Apple Watch are re...,1,22,False,0,0,1,0,0,0,1
15,15,b065262210783596c1fe79466b8f8985,"To start, can you tell us the software version...",1,17,False,1,0,0,0,0,1,0
16,16,b065262210783596c1fe79466b8f8985,"To start, can you tell us the software version...",1,17,False,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7102,7102,0d2116ef5bf6ffef241a8ecace87e633,@GloCare That's not true because till now the ...,1,13,False,0,1,0,0,0,0,1
7109,7109,0d2116ef5bf6ffef241a8ecace87e633,The issue is being resolved.,1,5,True,0,0,1,0,0,1,0
7118,7118,0d2116ef5bf6ffef241a8ecace87e633,@GloCare Now working..,1,3,True,0,0,1,0,0,0,1
7126,7126,17a981f02ad216cf55c609694e743beb,Glitches galore getting echo dot to connect to...,1,9,False,0,1,0,0,0,0,1


In [ ]:
filter_train=train.groupby(["sentence","is_salient"]).sample(frac=0.04, replace=False)


<ipython-input-48-7551bb4e91d7>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filter_train=filter_train.append(train[train["is_salient"]==1].groupby(["sentence"]).sample(frac=0.04, replace=False),ignore_index=True)


In [ ]:
filter_train=filter_train.append(train[train["is_salient"]==1].sample(frac=0.04))

<ipython-input-52-68ec1b207cde>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  filter_train=filter_train.append(train[train["is_salient"]==1].sample(frac=0.04))
<ipython-input-52-68ec1b207cde>:1: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  filter_train=filter_train.append(train[train["is_salient"]==1].sample(frac=0.04))


In [ ]:
filter_train

,Unnamed: 0,dialog_id,sentence,is_salient,word_count,is_question,annot_id_0,annot_id_1,annot_id_2,annot_id_3,annot_id_4,sender_agent,sender_customer
0,6759,6fd6f04d7d7eed52683aad3e66f6039b,!,0,1,False,0,0,1,0,0,0,1
1,1558,d370b1e57be01797211b2f13751c2238,1/2,0,1,False,0,0,1,0,0,1,0
2,3583,7f6f2953ae910482d9617d19b1680451,?,0,1,True,0,1,0,0,0,0,1
3,3624,7f6f2953ae910482d9617d19b1680451,?,0,1,False,0,0,1,0,0,0,1
4,160,f30d2bbc15e7ff32244761b38b67d160,Lewis,0,1,False,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2595,2595,440e951b7ebf31b2d4858dbcd1938a11,"@558516 Hi, kindly get a new replacement card ...",1,14,False,0,1,0,0,0,1,0
3248,3248,ee38567fa52f8c8cd41261a489c47114,Shall I cancel your reservation and process a ...,1,12,False,0,1,0,0,0,1,0
856,856,e7246842b6e3b27eaf5a3e9675a3d8f4,Keep getting error messages and the site keeps...,1,15,False,0,1,0,0,0,0,1
227,227,20327b84e8b6c205b7ad7e56b2861c66,@AirAsiaSupport i will go ahead with fresh boo...,1,20,False,0,1,0,0,0,0,1


In [ ]:
num_features=['word_count',
       'is_question', 'annot_id_0', 'annot_id_1', 'annot_id_2', 'annot_id_3',
       'annot_id_4', 'sender_agent', 'sender_customer']

In [ ]:
!pip install tensorflow --upgrade

In [ ]:
inputs = tokenizer(list(filter_train['sentence']), padding=True, truncation=True,return_tensors="tf")
outputs = model(inputs)

last_hidden_states = outputs.last_hidden_state

text_sequences = tokenizer(list(filter_train['sentence']), padding=True, truncation=True, max_length=100, return_tensors="tf")
outputs = model(**text_sequences)
last_hidden_states = outputs.last_hidden_state

In [ ]:
last_hidden_states.shape

TensorShape([65, 70, 768])

In [ ]:
# Extract numeric features from the dataframe
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')

numeric_features = filter_train[num_features].values
print(type(numeric_features))
# Concatenate the numeric features with the last hidden states
#numeric_features=tf.convert_to_tensor(numeric_features)
#concatenated_features = tf.concat([last_hidden_states, numeric_features], axis=1)
# Apply dropout and dense layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(last_hidden_states)
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Create the final model
model = tf.keras.Model(inputs=[text_input], outputs=[l])



<class 'numpy.ndarray'>


ValueError: ignored

In [ ]:
model.fit(filtered_train[features], filtered_train[eval], epochs=2)

ValueError: ignored